In [34]:
import pandas as pd

file_path = r"C:\Users\DELL\Desktop\Kariyer\Upwork\Customer_Segmentation_Analytics\data\online_retail_II.csv"

df = pd.read_csv(file_path, encoding='ISO-8859-1')  
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [35]:
df.shape

(1067371, 8)

In [36]:
df.isnull().mean().sort_values(ascending=False) * 100

Customer ID    22.766873
Description     0.410541
Country         0.000000
Price           0.000000
InvoiceDate     0.000000
Quantity        0.000000
StockCode       0.000000
Invoice         0.000000
dtype: float64

In [37]:
df.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

### ✅ Initial Observations

- Data Overview
The dataset contains 1,067,371 rows and 8 columns.
It includes details about products, prices, purchase dates, customer IDs, and countries.

- Missing Data
The ‘Customer ID’ column has ~22.77% missing values. I’ll need to decide whether to remove these rows or keep them depending on the analysis goal.
The ‘Description’ field also has ~0.41% missing values – not too critical, but something to keep in mind if we use product names in visualizations.

- Data Types
InvoiceDate is still in string format, so I’ll convert it to datetime.
Customer ID is a float, but it should be treated as a categorical or object type for segmentation.

In [38]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

In [39]:
df.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

In [40]:
# Time Variables
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Weekday'] = df['InvoiceDate'].dt.day_name()
df['Hour'] = df['InvoiceDate'].dt.hour 

In [41]:
df = df[df['Customer ID'].notna()].copy()

In [42]:
df['Revenue'] = df['Quantity'] * df['Price']

In [43]:
print(f"Total revenue: {df['Revenue'].sum():,.2f}")

Total revenue: 16,648,292.39


In [44]:
negative_revenue_count = df[df["Revenue"] <= 0].shape[0]
print(f"Number of rows with Revenue <= 0: {negative_revenue_count}")

df[df["Revenue"] <= 0].head()

Number of rows with Revenue <= 0: 18815


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Year,Month,Day,Weekday,Hour,Revenue
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia,2009,12,1,Tuesday,10,-35.4
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia,2009,12,1,Tuesday,10,-9.9
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia,2009,12,1,Tuesday,10,-17.0
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia,2009,12,1,Tuesday,10,-12.6
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia,2009,12,1,Tuesday,10,-35.4


In [45]:
df["TransactionType"] = df["Invoice"].astype(str).apply(lambda x: "Return" if x.startswith("C") else "Purchase")

In [46]:
df["TransactionType"].value_counts()

Purchase    805620
Return       18744
Name: TransactionType, dtype: int64

In [47]:
df['Customer ID'] = df['Customer ID'].astype(int)

In [48]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Year,Month,Day,Weekday,Hour,Revenue,TransactionType
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,2009,12,1,Tuesday,7,83.4,Purchase
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2009,12,1,Tuesday,7,81.0,Purchase
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2009,12,1,Tuesday,7,81.0,Purchase
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,2009,12,1,Tuesday,7,100.8,Purchase
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,2009,12,1,Tuesday,7,30.0,Purchase


In [49]:
import os
os.makedirs("../outputs", exist_ok=True)

In [50]:
df.to_csv("../outputs/cleaned_data.csv", index=False)

In [33]:
df.shape

(824364, 15)

### ✅ Data Cleaning Summary

- Removed records with missing `Customer ID`
- Converted `InvoiceDate` to datetime format
- Converted `Customer ID` to string type
- Created `Revenue` column as `Quantity × UnitPrice`
- Marked records as `Purchase` or `Return` based on `InvoiceNo`
- Final dataset contains: 1067371 rows × 15 columns
